In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/4월/나무문화_4월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

390


In [3]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_4월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,0,"['04', '01', '생일', '맞이', '강원도', '여행', '의', '시작...","[('04', 'Number'), ('01', 'Number'), ('생일', 'N..."
1,호텔 먼저 들어가서 체크인 선착순 티켓 받아오고 밥먹으러 고고싱 하마식당 웨이팅 걸...,0,"['호텔', '먼저', '들어가서', '체크', '인', '선착순', '티켓', '...","[('호텔', 'Noun'), ('먼저', 'Noun'), ('들어가다', 'Ver..."
2,진짜 바다도 너무 푸르고 그저 행복 잠깐 기다리면서 애들 찍어봤는데 뭔가 배경도 괜...,0,"['진짜', '바다', '도', '너무', '푸르고', '그저', '행복', '잠깐...","[('진짜', 'Noun'), ('바다', 'Noun'), ('도', 'Josa')..."
3,근데 진짜 날씨가 너무 좋아서ㅠㅠㅠ 완벽했던 여행 바다가 저리 푸르고 하늘은 구름한...,0,"['근데', '진짜', '날씨', '가', '너무', '좋아서', 'ㅠㅠㅠ', '완...","[('근데', 'Adverb'), ('진짜', 'Noun'), ('날씨', 'Nou..."
4,그리고 바닷가에서 벗어나니까 막 춥진 않은 날씨라서 식당 앞에서 기다리고 딱 좋음,0,"['그리고', '바닷가', '에서', '벗어나니까', '막', '춥진', '않은',...","[('그리고', 'Conjunction'), ('바닷가', 'Noun'), ('에서..."
...,...,...,...,...
821100,분석글은 저의 투자시점과 생각이고 흐름만 파악하는 용도이지 주식을 추천하는건 아닙니다,18280,"['분석', '글', '은', '저', '의', '투자', '시점', '과', '생...","[('분석', 'Noun'), ('글', 'Noun'), ('은', 'Josa'),..."
821101,참고 만 하시고 투자손실의 책임은 본인에 있습니다,18280,"['참고', '만', '하시고', '투자', '손실', '의', '책임', '은',...","[('참고', 'Noun'), ('만', 'Noun'), ('하다', 'Verb')..."
821102,같이보면 좋은 포스팅 국내주식 사료 관련주 안녕하세요 여러분 주말에 푹쉬고 헬요일 ...,18280,"['같이', '보면', '좋은', '포스팅', '국내', '주식', '사료', '관...","[('같이', 'Adverb'), ('보다', 'Verb'), ('좋다', 'Adj..."
821103,오늘 정리해볼 테마는 사료관련주입니다 사료 m blog naver com 임산부커피...,18280,"['오늘', '정리', '해볼', '테마', '는', '사료', '관련', '주입'...","[('오늘', 'Noun'), ('정리', 'Noun'), ('해보다', 'Verb..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs
0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,0,"['04', '01', '생일', '맞이', '강원도', '여행', '의', '시작...","[('04', 'Number'), ('01', 'Number'), ('생일', 'N...",04 01 생일 맞이 강원도 여행 의 시작 후후 제일 기대하던 여행 하지만 원래 같...
1,호텔 먼저 들어가서 체크인 선착순 티켓 받아오고 밥먹으러 고고싱 하마식당 웨이팅 걸...,0,"['호텔', '먼저', '들어가서', '체크', '인', '선착순', '티켓', '...","[('호텔', 'Noun'), ('먼저', 'Noun'), ('들어가다', 'Ver...",호텔 먼저 들어가서 체크 인 선착순 티켓 받아오고 밥 먹으러 고고싱 하마 식당 웨이...
2,진짜 바다도 너무 푸르고 그저 행복 잠깐 기다리면서 애들 찍어봤는데 뭔가 배경도 괜...,0,"['진짜', '바다', '도', '너무', '푸르고', '그저', '행복', '잠깐...","[('진짜', 'Noun'), ('바다', 'Noun'), ('도', 'Josa')...",진짜 바다 도 너무 푸르고 그저 행복 잠깐 기다리면서 애 들 찍어 봤는데 뭔가 배경...
3,근데 진짜 날씨가 너무 좋아서ㅠㅠㅠ 완벽했던 여행 바다가 저리 푸르고 하늘은 구름한...,0,"['근데', '진짜', '날씨', '가', '너무', '좋아서', 'ㅠㅠㅠ', '완...","[('근데', 'Adverb'), ('진짜', 'Noun'), ('날씨', 'Nou...",근데 진짜 날씨 가 너무 좋아서 ㅠㅠㅠ 완벽했던 여행 바다 가 저리 푸르고 하늘 은...
4,그리고 바닷가에서 벗어나니까 막 춥진 않은 날씨라서 식당 앞에서 기다리고 딱 좋음,0,"['그리고', '바닷가', '에서', '벗어나니까', '막', '춥진', '않은',...","[('그리고', 'Conjunction'), ('바닷가', 'Noun'), ('에서...",그리고 바닷가 에서 벗어나니까 막 춥진 않은 날씨 라서 식당 앞 에서 기다리고 딱 좋음
...,...,...,...,...,...
821100,분석글은 저의 투자시점과 생각이고 흐름만 파악하는 용도이지 주식을 추천하는건 아닙니다,18280,"['분석', '글', '은', '저', '의', '투자', '시점', '과', '생...","[('분석', 'Noun'), ('글', 'Noun'), ('은', 'Josa'),...",분석 글 은 저 의 투자 시점 과 생각 이고 흐름 만 파악 하는 용도 이지 주식 을...
821101,참고 만 하시고 투자손실의 책임은 본인에 있습니다,18280,"['참고', '만', '하시고', '투자', '손실', '의', '책임', '은',...","[('참고', 'Noun'), ('만', 'Noun'), ('하다', 'Verb')...",참고 만 하시고 투자 손실 의 책임 은 본인 에 있습니다
821102,같이보면 좋은 포스팅 국내주식 사료 관련주 안녕하세요 여러분 주말에 푹쉬고 헬요일 ...,18280,"['같이', '보면', '좋은', '포스팅', '국내', '주식', '사료', '관...","[('같이', 'Adverb'), ('보다', 'Verb'), ('좋다', 'Adj...",같이 보면 좋은 포스팅 국내 주식 사료 관련 주 안녕하세요 여러분 주말 에 푹쉬고 ...
821103,오늘 정리해볼 테마는 사료관련주입니다 사료 m blog naver com 임산부커피...,18280,"['오늘', '정리', '해볼', '테마', '는', '사료', '관련', '주입'...","[('오늘', 'Noun'), ('정리', 'Noun'), ('해보다', 'Verb...",오늘 정리 해볼 테마 는 사료 관련 주입 니 다 사료 m blog naver com...


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,0,"['04', '01', '생일', '맞이', '강원도', '여행', '의', '시작...","[('04', 'Number'), ('01', 'Number'), ('생일', 'N...",04 01 생일 맞이 강원도 여행 의 시작 후후 제일 기대하던 여행 하지만 원래 같...,04 01 생일 맞이 강원도 여행 의 시작 후후 제일 기대하던 여행 하지만 원래 같...
1,호텔 먼저 들어가서 체크인 선착순 티켓 받아오고 밥먹으러 고고싱 하마식당 웨이팅 걸...,0,"['호텔', '먼저', '들어가서', '체크', '인', '선착순', '티켓', '...","[('호텔', 'Noun'), ('먼저', 'Noun'), ('들어가다', 'Ver...",호텔 먼저 들어가서 체크 인 선착순 티켓 받아오고 밥 먹으러 고고싱 하마 식당 웨이...,04 01 생일 맞이 강원도 여행 의 시작 후후 제일 기대하던 여행 하지만 원래 같...
2,진짜 바다도 너무 푸르고 그저 행복 잠깐 기다리면서 애들 찍어봤는데 뭔가 배경도 괜...,0,"['진짜', '바다', '도', '너무', '푸르고', '그저', '행복', '잠깐...","[('진짜', 'Noun'), ('바다', 'Noun'), ('도', 'Josa')...",진짜 바다 도 너무 푸르고 그저 행복 잠깐 기다리면서 애 들 찍어 봤는데 뭔가 배경...,04 01 생일 맞이 강원도 여행 의 시작 후후 제일 기대하던 여행 하지만 원래 같...
3,근데 진짜 날씨가 너무 좋아서ㅠㅠㅠ 완벽했던 여행 바다가 저리 푸르고 하늘은 구름한...,0,"['근데', '진짜', '날씨', '가', '너무', '좋아서', 'ㅠㅠㅠ', '완...","[('근데', 'Adverb'), ('진짜', 'Noun'), ('날씨', 'Nou...",근데 진짜 날씨 가 너무 좋아서 ㅠㅠㅠ 완벽했던 여행 바다 가 저리 푸르고 하늘 은...,04 01 생일 맞이 강원도 여행 의 시작 후후 제일 기대하던 여행 하지만 원래 같...
4,그리고 바닷가에서 벗어나니까 막 춥진 않은 날씨라서 식당 앞에서 기다리고 딱 좋음,0,"['그리고', '바닷가', '에서', '벗어나니까', '막', '춥진', '않은',...","[('그리고', 'Conjunction'), ('바닷가', 'Noun'), ('에서...",그리고 바닷가 에서 벗어나니까 막 춥진 않은 날씨 라서 식당 앞 에서 기다리고 딱 좋음,04 01 생일 맞이 강원도 여행 의 시작 후후 제일 기대하던 여행 하지만 원래 같...
...,...,...,...,...,...,...
821100,분석글은 저의 투자시점과 생각이고 흐름만 파악하는 용도이지 주식을 추천하는건 아닙니다,18280,"['분석', '글', '은', '저', '의', '투자', '시점', '과', '생...","[('분석', 'Noun'), ('글', 'Noun'), ('은', 'Josa'),...",분석 글 은 저 의 투자 시점 과 생각 이고 흐름 만 파악 하는 용도 이지 주식 을...,안녕하세요 여러분 오늘 정리 해볼 테마 는 육계 닭고기 관련 주 정리 해볼건데 요 ...
821101,참고 만 하시고 투자손실의 책임은 본인에 있습니다,18280,"['참고', '만', '하시고', '투자', '손실', '의', '책임', '은',...","[('참고', 'Noun'), ('만', 'Noun'), ('하다', 'Verb')...",참고 만 하시고 투자 손실 의 책임 은 본인 에 있습니다,안녕하세요 여러분 오늘 정리 해볼 테마 는 육계 닭고기 관련 주 정리 해볼건데 요 ...
821102,같이보면 좋은 포스팅 국내주식 사료 관련주 안녕하세요 여러분 주말에 푹쉬고 헬요일 ...,18280,"['같이', '보면', '좋은', '포스팅', '국내', '주식', '사료', '관...","[('같이', 'Adverb'), ('보다', 'Verb'), ('좋다', 'Adj...",같이 보면 좋은 포스팅 국내 주식 사료 관련 주 안녕하세요 여러분 주말 에 푹쉬고 ...,안녕하세요 여러분 오늘 정리 해볼 테마 는 육계 닭고기 관련 주 정리 해볼건데 요 ...
821103,오늘 정리해볼 테마는 사료관련주입니다 사료 m blog naver com 임산부커피...,18280,"['오늘', '정리', '해볼', '테마', '는', '사료', '관련', '주입'...","[('오늘', 'Noun'), ('정리', 'Noun'), ('해보다', 'Verb...",오늘 정리 해볼 테마 는 사료 관련 주입 니 다 사료 m blog naver com...,안녕하세요 여러분 오늘 정리 해볼 테마 는 육계 닭고기 관련 주 정리 해볼건데 요 ...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,04 01 생일 맞이 강원도 여행 의 시작 후후 제일 기대하던 여행 하지만 원래 같...
1,1,드라이브 코스 카페 귀 천 천상병 시인 을 알아보는 곳 위치 영업일 휴무 일 메뉴 ...
2,2,안녕하세요 항상 화 eating 하는 카림 1 기 의 오 지선 입니다 튀니지 여행 ...
3,3,안녕하세요 도도한 쑤 언니 입니다 D 작년 에 순천 선암사 로 겹 벚꽃 을 구경 하...
4,4,새벽 에 쓰다가 한번 날렸습니다 네네 4월 도 끝났 네 여 참 한 달 한 달 느린거...
...,...,...
18276,18276,블로그 게시판 72 개 의 글 게시판 목록 열기 게시판 스타일 라디오 최저 가 공구...
18277,18277,영화 엽기 적 인 그녀 에서 어떤 인물 을 말 할 때 엽기 적이다고 하는 가 한자 ...
18278,18278,2022년 은 더 좋은 사람 들 과 더 많은 꽃 구경 을 다닐 수 있도록 울산 벚꽃...
18279,18279,3월 이 다 지나가고 전시 도 막 을 내렸지만 블로그 에 포스팅 하는 이유 는 전시...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(18281, 422464)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.02064257, 0.01144133, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0238785 , 0.05514533, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00631263, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03379986, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,422454,422455,422456,422457,422458,422459,422460,422461,422462,422463
0,0.020643,0.011441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.023878,0.055145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.006313,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.065781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.020827,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18276,0.005954,0.011000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18277,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18278,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18279,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0001,00010,0001064,00011,0001151675,0001153056,00012,...,힛트,힛트곡,힛트쏭,힜다,힜습니,힝,힝구,힝힝,힣,힣힣
0,0.020643,0.011441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.023878,0.055145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.006313,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.065781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.020827,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18276,0.005954,0.011000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18277,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18278,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18279,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/4월/나무문화_4월_TF-IDF행렬(원본).csv", index=False, encoding='UTF-8')

In [ ]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

In [49]:
# 한글로만 이루어진 TF-IDF 행렬도 txt로 저장해줌.
korean_df.to_csv("../dataset/2022/본문_TF-IDF/4월/나무문화_4월_TF-IDF행렬(원본).txt", index=False, encoding='UTF-8')

In [50]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,51.032181
1,00,50.503403
2,000,19.454386
3,0001,0.049739
4,0001076717,0.034615
...,...,...
134284,힙니,0.062174
134285,힙스터,0.058074
134286,힙합,0.405733
134287,힝,0.086247


In [51]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/1월/나무문화_1월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')